Copyright 2025 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Day 1 - Evaluation and structured output

## Overview

Welcome back to the Kaggle 5-day Generative AI course. In this notebook, you'll learn some techniques for evaluating the output of a language model. As part of the evaluation, you will also use Gemini's structured data capability to produce evaluation results as instances of Python types.

Note: This notebook is more code-heavy than the first Day 1 notebook ([Prompting](https://www.kaggle.com/code/markishere/day-1-prompting/)). This notebook is not a prerequisite for days 2 and beyond, so feel free to skip over it, or come back later in the week. If you have not yet tried the [Prompting](https://www.kaggle.com/code/markishere/day-1-prompting/) notebook, start there first as it introduces the fundamentals for interacting with LLMs.

Also check out the **bonus whitepaper** on [Evaluating Large Language Models](https://services.google.com/fh/files/blogs/neurips_evaluation.pdf).

## For help

**Common issues are covered in the [FAQ and troubleshooting guide](https://www.kaggle.com/code/markishere/day-0-troubleshooting-and-faqs).**


## Setup

Install the Python SDK.

In [2]:
!pip install -Uq "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.3 MB/s eta 0:00:00


In [3]:
from google import genai
from google.genai import types

from IPython.display import Markdown, display

genai.__version__

'1.7.0'

### Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [4]:
from kaggle_secrets import UserSecretsClient

client = genai.Client(api_key=UserSecretsClient().get_secret("GOOGLE_API_KEY"))

If you received an error response along the lines of `No user secrets exist for kernel id ...`, then you need to add your API key via `Add-ons`, `Secrets` **and** enable it.

![Screenshot of the checkbox to enable GOOGLE_API_KEY secret](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_3.png)

### Automated retry

This codelab sends a lot of requests, so set up an automatic retry
that ensures your requests are retried when per-minute quota is reached.

In [5]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

## Evaluation

When using LLMs in real-world cases, it's important to understand how well they are performing. The open-ended generation capabilities of LLMs can make many cases difficult to measure. In this notebook you will walk through some simple techniques for evaluating LLM outputs and understanding their performance.

For this example, you'll evaluate a summarisation task using the [Gemini 1.5 Pro technical report](https://storage.googleapis.com/cloud-samples-data/generative-ai/pdf/2403.05530.pdf). Start by downloading the PDF to the notebook environment, and uploading that copy for use with the Gemini API.

In [6]:
!wget -nv -O gemini.pdf https://storage.googleapis.com/cloud-samples-data/generative-ai/pdf/2403.05530.pdf

document_file = client.files.upload(file='gemini.pdf')

2025-03-31 16:18:45 URL:https://storage.googleapis.com/cloud-samples-data/generative-ai/pdf/2403.05530.pdf [7228817/7228817] -> "gemini.pdf" [1]


### Summarise a document

The summarisation request used here is fairly basic. It targets the training content specifically but provides no guidance otherwise.

In [7]:
request = 'Tell me about the training process used here.'

def summarise_doc(request: str) -> str:
  """Execute the request on the uploaded document."""
  # Set the temperature low to stabilise the output.
  config = types.GenerateContentConfig(temperature=0.0)
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=config,
      contents=[request, document_file],
  )

  return response.text

summary = summarise_doc(request)
Markdown(summary)

Certainly! Let's break down the training process used for Gemini 1.5 Pro, based on the information provided in the document.

**Key Aspects of the Training Process:**

1.  **Model Architecture:**
    *   Gemini 1.5 Pro is a sparse Mixture-of-Experts (MoE) Transformer-based model.
    *   It builds upon the research advances of Gemini 1.0 and a longer history of MoE research at Google.
    *   MoE models use a learned routing function to direct inputs to a subset of the model's parameters for processing. This allows for a larger total parameter count while keeping the number of activated parameters constant.

2.  **Training Infrastructure:**
    *   Trained on multiple 4096-chip pods of Google's TPUv4 accelerators.
    *   The training is distributed across multiple datacenters.

3.  **Training Data:**
    *   A variety of multimodal and multilingual data is used.
    *   The pre-training dataset includes data sourced from various domains, including web documents, code, images, audio, and video content.

4.  **Instruction Tuning:**
    *   Gemini 1.5 Pro is fine-tuned on a collection of multimodal data containing paired instructions and appropriate responses.
    *   Further tuning is based on human preference data.

5.  **Long-Context Understanding:**
    *   The model incorporates significant architecture changes to enable long-context understanding of inputs up to 10 million tokens without degrading performance.

6.  **Safety Mitigations:**
    *   Supervised fine-tuning (SFT) and reinforcement learning through human feedback (RLHF) are used to mitigate safety risks.
    *   The safety mitigation is focused on adversarial or "harm-inducing" queries.
    *   New image-to-text SFT data is incorporated, as safety SFT data for text-only queries was not as effective for harm-inducing image-to-text queries.

**In summary:** The training process for Gemini 1.5 Pro involves a combination of a sophisticated MoE Transformer architecture, massive computational resources, diverse multimodal data, instruction tuning, and safety mitigations. The model is designed to handle extremely long contexts, enabling it to process and reason over large amounts of information from various modalities.

### Define an evaluator

For a task like this, you may wish to evaluate a number of aspects, like how well the model followed the prompt ("instruction following"), whether it included relevant data in the prompt ("groundedness"), how easy the text is to read ("fluency"), or other factors like "verbosity" or "quality".

You can instruct an LLM to perform these tasks in a similar manner to how you would instruct a human rater: with a clear definition and [assessment rubric](https://en.wikipedia.org/wiki/Rubric_%28academic%29).

In this step, you define an evaluation agent using a pre-written "summarisation" prompt and use it to gauge the quality of the generated summary.

Note: For more pre-written evaluation prompts covering groundedness, safety, coherence and more, check out this [comprehensive list of model-based evaluation prompts](https://cloud.google.com/vertex-ai/generative-ai/docs/models/metrics-templates) from the Google Cloud docs.

In [8]:
import enum

# Define the evaluation prompt
SUMMARY_PROMPT = """\
# Instruction
You are an expert evaluator. Your task is to evaluate the quality of the responses generated by AI models.
We will provide you with the user input and an AI-generated responses.
You should first read the user input carefully for analyzing the task, and then evaluate the quality of the responses based on the Criteria provided in the Evaluation section below.
You will assign the response a rating following the Rating Rubric and Evaluation Steps. Give step-by-step explanations for your rating, and only choose ratings from the Rating Rubric.

# Evaluation
## Metric Definition
You will be assessing summarization quality, which measures the overall ability to summarize text. Pay special attention to length constraints, such as in X words or in Y sentences. The instruction for performing a summarization task and the context to be summarized are provided in the user prompt. The response should be shorter than the text in the context. The response should not contain information that is not present in the context.

## Criteria
Instruction following: The response demonstrates a clear understanding of the summarization task instructions, satisfying all of the instruction's requirements.
Groundedness: The response contains information included only in the context. The response does not reference any outside information.
Conciseness: The response summarizes the relevant details in the original text without a significant loss in key information without being too verbose or terse.
Fluency: The response is well-organized and easy to read.

## Rating Rubric
5: (Very good). The summary follows instructions, is grounded, is concise, and fluent.
4: (Good). The summary follows instructions, is grounded, concise, and fluent.
3: (Ok). The summary mostly follows instructions, is grounded, but is not very concise and is not fluent.
2: (Bad). The summary is grounded, but does not follow the instructions.
1: (Very bad). The summary is not grounded.

## Evaluation Steps
STEP 1: Assess the response in aspects of instruction following, groundedness, conciseness, and verbosity according to the criteria.
STEP 2: Score based on the rubric.

# User Inputs and AI-generated Response
## User Inputs

### Prompt
{prompt}

## AI-generated Response
{response}
"""

# Define a structured enum class to capture the result.
class SummaryRating(enum.Enum):
  VERY_GOOD = '5'
  GOOD = '4'
  OK = '3'
  BAD = '2'
  VERY_BAD = '1'


def eval_summary(prompt, ai_response):
  """Evaluate the generated summary against the prompt used."""

  chat = client.chats.create(model='gemini-2.0-flash')

  # Generate the full text response.
  response = chat.send_message(
      message=SUMMARY_PROMPT.format(prompt=prompt, response=ai_response)
  )
  verbose_eval = response.text

  # Coerce into the desired structure.
  structured_output_config = types.GenerateContentConfig(
      response_mime_type="application/json",
      response_schema=SummaryRating,
  )
  response = chat.send_message(
      message="Convert the final score.",
      config=structured_output_config,
  )
  structured_eval = response.parsed

  return verbose_eval, structured_eval


text_eval, struct_eval = eval_summary(prompt=[request, document_file], ai_response=summary)
Markdown(text_eval)

## Evaluation

### STEP 1: Assess the response in aspects of instruction following, groundedness, conciseness, and verbosity according to the criteria.
Instruction Following: The response does not explicitly follow instructions. I only asked for information about the training process.
Groundedness: The response is grounded.
Conciseness: The response is concise.
Fluency: The response is fluent.

### STEP 2: Score based on the rubric.
3

In this example, the model generated a textual justification that was set up in a chat context. This full text response is useful both for human interpretation and for giving the model a place to "collect notes" while it assesses the text and produces a final score. This "note taking" or "thinking" strategy typically works well with auto-regressive models, where the generated text is passed back into the model at each generation step. This means the working "notes" are used when generating final result output.

In the next turn, the model converts the text output into a structured response. If you want to aggregate scores or use them programatically then you want to avoid parsing the unstructured text output. Here the `SummaryRating` schema is passed, so the model converts the chat history into an instance of the `SummaryRating` enum.

In [9]:
struct_eval

<SummaryRating.OK: '3'>

### Make the summary prompt better or worse

Gemini models tend to be quite good at tasks like direct summarisation without much prompting, so you should expect to see a result like `GOOD` or `VERY_GOOD` on the previous task, even with a rudimentary prompt. Run it a few times to get a feel for the average response.

To explore how to influence the summarisation output, consider what you might change in the summary request prompt to change the result. Take a look at the evaluation `SUMMARY_PROMPT` for some ideas.

Try the following tweaks and see how they positively or negatively change the result:
* Be specific with the size of the summary,
* Request specific information,
* Ask about information that is not in the document,
* Ask for different degrees of summarisation (such as "explain like I'm 5" or "with full technical depth")

In [10]:
new_prompt = "What is the best LLM?"
# Try:
#  ELI5 the training process
#  Summarise the needle/haystack evaluation technique in 1 line
#  Describe the model architecture to someone with a civil engineering degree
#  What is the best LLM?

if not new_prompt:
  raise ValueError("Try setting a new summarisation prompt.")


def run_and_eval_summary(prompt):
  """Generate and evaluate the summary using the new prompt."""
  summary = summarise_doc(new_prompt)
  display(Markdown(summary + '\n-----'))

  text, struct = eval_summary([new_prompt, document_file], summary)
  display(Markdown(text + '\n-----'))
  print(struct)

run_and_eval_summary(new_prompt)

Determining the "best" LLM is complex because it depends on the specific task, desired characteristics (like speed, cost, or context window), and the benchmarks used for evaluation.  There isn't a single, universally agreed-upon "best" model.

However, based on the provided document, here's a summary of the key points about Gemini 1.5 Pro and its position relative to other models:

*   **Gemini 1.5 Pro:** This is the latest model from the Gemini family, developed by Google DeepMind. It's a multimodal mixture-of-experts model.

*   **Key Features of Gemini 1.5 Pro:**

    *   **Long Context:** It excels at recalling and reasoning over fine-grained information from millions of tokens of context (up to 10M tokens and beyond). This is a significant leap compared to models like Claude 2.1 (200k) and GPT-4 Turbo (128k).
    *   **Multimodal:** It can handle various input modalities, including text, video, and audio.
    *   **Efficiency:** It achieves performance comparable to Gemini 1.0 Ultra (a state-of-the-art model) while requiring significantly less compute to train.
    *   **In-Context Learning:** It demonstrates surprising new capabilities, such as learning to translate a new language (Kalamang) from a single set of linguistic documentation.
    *   **Strong Performance:** It achieves near-perfect recall on long-context retrieval tasks, improves state-of-the-art in long-document QA, long-video QA, and long-context ASR, and matches or surpasses Gemini 1.0 Ultra's performance across a broad set of benchmarks.

*   **Gemini 1.5 Pro vs. Other Models:**

    *   **Gemini 1.0 Pro:** Gemini 1.5 Pro generally outperforms Gemini 1.0 Pro across a wide range of benchmarks.
    *   **Gemini 1.0 Ultra:** Gemini 1.5 Pro performs better than Gemini 1.0 Ultra on more than half of the benchmarks, despite using less training compute.
    *   **Claude 2.1:** Gemini 1.5 Pro significantly surpasses Claude 2.1 in long-context recall.
    *   **GPT-4 Turbo:** Gemini 1.5 Pro demonstrates superior long-context capabilities compared to GPT-4 Turbo.
    *   **GPT-4V:** Gemini 1.5 Pro outperforms GPT-4V on video-related tasks, especially with longer video inputs.

*   **Limitations and Considerations:**

    *   The document is a technical report from Google DeepMind, so it presents a favorable view of Gemini 1.5 Pro.
    *   The "best" model depends on the specific task. While Gemini 1.5 Pro excels in long-context scenarios, other models might be more suitable for tasks with shorter context requirements.
    *   The document acknowledges that the choice of prompting method and type of needle affect final performance of models, and future versions of “needle(s)-in-a-haystack" style tests should account for prompt robustness.
    *   The document acknowledges that the model struggles to understand when to discuss contextually relevant information.

**In conclusion:**

If your primary need is for a model that can process and reason over extremely long contexts (millions of tokens) and handle multiple modalities, Gemini 1.5 Pro appears to be a leading contender based on this document. However, it's essential to consider your specific use case, evaluate other models, and be aware of the potential limitations.
-----

STEP 1: Assess the response in aspects of instruction following, groundedness, conciseness, and verbosity according to the criteria.

Instruction Following: The model successfully identifies the important and relevant information in the context and returns this information in the response. The model followed the instructions well.
Groundedness: The response is grounded in the context provided. There is no outside information used in the response.
Conciseness: The response is not very concise. It is verbose, although it is a summary of the document provided.
Fluency: The response is well-written and easy to read.

STEP 2: Score based on the rubric.

4: (Good). The summary follows instructions, is grounded, concise, and fluent.
-----

SummaryRating.GOOD


In [11]:
new_prompt = "Describe the model architecture to someone with a civil engineering degree"
# Try:
#  ELI5 the training process
#  Summarise the needle/haystack evaluation technique in 1 line
#  Describe the model architecture to someone with a civil engineering degree
#  What is the best LLM?

if not new_prompt:
  raise ValueError("Try setting a new summarisation prompt.")


def run_and_eval_summary(prompt):
  """Generate and evaluate the summary using the new prompt."""
  summary = summarise_doc(new_prompt)
  display(Markdown(summary + '\n-----'))

  text, struct = eval_summary([new_prompt, document_file], summary)
  display(Markdown(text + '\n-----'))
  print(struct)

run_and_eval_summary(new_prompt)

Okay, here's a breakdown of the Gemini 1.5 Pro model architecture, tailored for someone with a civil engineering background:

**The Big Picture: A System of Systems**

Think of Gemini 1.5 Pro not as a single monolithic structure, but as a complex system of interconnected components, much like a large-scale infrastructure project.  It's designed to process and understand information from various sources (text, images, audio, video) and then use that understanding to perform tasks.

**Key Components (Like Structural Elements):**

1.  **Transformer Architecture (The Foundation):**
    *   At its core, Gemini 1.5 Pro is based on the "Transformer" architecture.  Imagine this as the underlying framework, like the reinforced concrete foundation of a building.
    *   Transformers are good at finding relationships between different parts of the input data, no matter how far apart they are. This is crucial for understanding long documents or videos where context is important.
    *   The Transformer uses a mechanism called "attention" to focus on the most relevant parts of the input when making decisions. Think of it as selectively reinforcing certain areas of the foundation based on the load they're carrying.

2.  **Mixture-of-Experts (MoE) (The Load-Bearing Walls):**
    *   This is where Gemini 1.5 Pro gets its efficiency and scalability. Instead of having one giant neural network, it uses a "mixture of experts."
    *   Imagine a building with different specialized teams (experts) for different tasks (e.g., one team for text analysis, another for image recognition, another for audio processing).
    *   A "routing function" acts like a project manager, deciding which expert team is best suited to handle a particular piece of the input data. This is like assigning the right crew to the right task on a construction site.
    *   The key benefit is that not all the "experts" are active at the same time. This significantly reduces the computational cost, allowing the model to be much larger and handle more complex tasks without requiring excessive resources.

3.  **Multimodal Input Processing (The Utility Connections):**
    *   Gemini 1.5 Pro is designed to handle multiple types of data (text, images, audio, video) simultaneously.
    *   Think of this as the utility connections (electricity, water, data) coming into the building. Each type of data is processed by specialized modules that convert it into a common format that the Transformer can understand.
    *   This allows the model to understand the relationships between different modalities. For example, it can understand that a caption describes an image or that a sound effect is associated with a particular scene in a video.

4.  **Long Context Window (The Span of the Structure):**
    *   This is a key innovation of Gemini 1.5 Pro. It can process extremely long sequences of data (millions of "tokens," which are essentially words or parts of words).
    *   Think of this as the overall span of a bridge or the length of a tunnel. The longer the span, the more complex the engineering challenges.
    *   The long context window allows the model to understand the overall structure and meaning of very large inputs, such as entire books, hours of video, or large codebases.

**How It Works (The Construction Process):**

1.  **Input:** The model receives input data (text, image, audio, video, or a combination).
2.  **Modality Processing:** Each modality is processed by its respective module to convert it into a suitable format.
3.  **Routing:** The routing function decides which "experts" are best suited to handle the input.
4.  **Processing:** The selected experts process the input data using the Transformer architecture.
5.  **Output:** The model generates an output based on its understanding of the input.

**Key Advantages (The Benefits of the Design):**

*   **Efficiency:** The MoE architecture allows the model to be very large and powerful without requiring excessive computational resources.
*   **Scalability:** The model can handle extremely long sequences of data.
*   **Multimodality:** The model can understand and reason about information from various sources.
*   **Generalization:** The model can perform a wide range of tasks, from answering questions to translating languages.

**In Summary:**

Gemini 1.5 Pro is a sophisticated system that combines the power of the Transformer architecture with the efficiency of a mixture-of-experts approach. Its ability to handle multimodal input and extremely long sequences of data makes it a powerful tool for understanding and processing complex information. Think of it as a well-designed and efficiently constructed infrastructure project that can handle a wide range of demands.

-----

## Evaluation
STEP 1: The response is well organized and written for the specified audience. It provides an explanation of the model architecture in a way that a civil engineer would understand.
STEP 2: I rated this as a 5 because it followed the instruction, was grounded, and fluent.

## Rating:
5

-----

SummaryRating.VERY_GOOD


## Evaluating in practice

Evaluation has many practical uses, for example:
* You can quickly iterate on a prompt with a small set of test documents,
* You can compare different models to find what works best for your needs, such as finding the trade-off between price and performance, or finding the best performance for a specific task.
* When pushing changes to a model or prompt in a production system, you can verify that the system does not regress in quality.

In this section you will try two different evaluation approaches.

### Pointwise evaluation

The technique used above, where you evaluate a single input/output pair against some criteria is known as pointwise evaluation. This is useful for evaluating singular outputs in an absolute sense, such as "was it good or bad?"

In this exercise, you will try different guidance prompts with a set of questions.

In [12]:
import functools

# Try these instructions, or edit and add your own.
terse_guidance = "Answer the following question in a single sentence, or as close to that as possible."
moderate_guidance = "Provide a brief answer to the following question, use a citation if necessary, but only enough to answer the question."
cited_guidance = "Provide a thorough, detailed answer to the following question, citing the document and supplying additional background information as much as possible."
guidance_options = {
    'Terse': terse_guidance,
    'Moderate': moderate_guidance,
    'Cited': cited_guidance,
}

questions = [
    # Un-comment one or more questions to try here, or add your own.
    # Evaluating more questions will take more time, but produces results
    # with higher confidence. In a production system, you may have hundreds
    # of questions to evaluate a complex system.

    # "What metric(s) are used to evaluate long context performance?",
    # "How does the model perform on code tasks?",
    "How many layers does it have?",
    # "Why is it called Gemini?",
]

if not questions:
  raise NotImplementedError('Add some questions to evaluate!')


@functools.cache
def answer_question(question: str, guidance: str = '') -> str:
  """Generate an answer to the question using the uploaded document and guidance."""
  config = types.GenerateContentConfig(
      temperature=0.0,
      system_instruction=guidance,
  )
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=config,
      contents=[question, document_file],
  )

  return response.text


answer = answer_question(questions[0], terse_guidance)
Markdown(answer)

Gemini 1.5 Pro is a sparse mixture-of-expert (MoE) Transformer-based model.

Now set up a question-answering evaluator, much like before, but using the [pointwise QA evaluation prompt](https://cloud.google.com/vertex-ai/generative-ai/docs/models/metrics-templates#pointwise_question_answering_quality).

In [13]:
import enum

QA_PROMPT = """\
# Instruction
You are an expert evaluator. Your task is to evaluate the quality of the responses generated by AI models.
We will provide you with the user prompt and an AI-generated responses.
You should first read the user prompt carefully for analyzing the task, and then evaluate the quality of the responses based on and rules provided in the Evaluation section below.

# Evaluation
## Metric Definition
You will be assessing question answering quality, which measures the overall quality of the answer to the question in the user prompt. Pay special attention to length constraints, such as in X words or in Y sentences. The instruction for performing a question-answering task is provided in the user prompt. The response should not contain information that is not present in the context (if it is provided).

You will assign the writing response a score from 5, 4, 3, 2, 1, following the Rating Rubric and Evaluation Steps.
Give step-by-step explanations for your scoring, and only choose scores from 5, 4, 3, 2, 1.

## Criteria Definition
Instruction following: The response demonstrates a clear understanding of the question answering task instructions, satisfying all of the instruction's requirements.
Groundedness: The response contains information included only in the context if the context is present in the user prompt. The response does not reference any outside information.
Completeness: The response completely answers the question with sufficient detail.
Fluent: The response is well-organized and easy to read.

## Rating Rubric
5: (Very good). The answer follows instructions, is grounded, complete, and fluent.
4: (Good). The answer follows instructions, is grounded, complete, but is not very fluent.
3: (Ok). The answer mostly follows instructions, is grounded, answers the question partially and is not very fluent.
2: (Bad). The answer does not follow the instructions very well, is incomplete or not fully grounded.
1: (Very bad). The answer does not follow the instructions, is wrong and not grounded.

## Evaluation Steps
STEP 1: Assess the response in aspects of instruction following, groundedness,completeness, and fluency according to the criteria.
STEP 2: Score based on the rubric.

# User Inputs and AI-generated Response
## User Inputs
### Prompt
{prompt}

## AI-generated Response
{response}
"""

class AnswerRating(enum.Enum):
  VERY_GOOD = '5'
  GOOD = '4'
  OK = '3'
  BAD = '2'
  VERY_BAD = '1'


@functools.cache
def eval_answer(prompt, ai_response, n=1):
  """Evaluate the generated answer against the prompt/question used."""
  chat = client.chats.create(model='gemini-2.0-flash')

  # Generate the full text response.
  response = chat.send_message(
      message=QA_PROMPT.format(prompt=[prompt, document_file], response=ai_response)
  )
  verbose_eval = response.text

  # Coerce into the desired structure.
  structured_output_config = types.GenerateContentConfig(
      response_mime_type="application/json",
      response_schema=AnswerRating,
  )
  response = chat.send_message(
      message="Convert the final score.",
      config=structured_output_config,
  )
  structured_eval = response.parsed

  return verbose_eval, structured_eval


text_eval, struct_eval = eval_answer(prompt=questions[0], ai_response=answer)
display(Markdown(text_eval))
print(struct_eval)

STEP 1: The AI model did not specify how many layers Gemini 1.5 Pro has. The response just explains the type of model.
STEP 2: The answer does not follow the instructions very well, is incomplete or not fully grounded, therefore the score is 2.

Score: 2


AnswerRating.BAD


Now run the evaluation task in a loop. Note that the guidance instruction is hidden from the evaluation agent. If you passed the guidance prompt, the model would score based on whether it followed that guidance, but for this task the goal is to find the best overall result based on the user's question, not the developers instruction.

In [14]:
import collections
import itertools

# Number of times to repeat each task in order to reduce error and calculate an average.
# Increasing it will take longer but give better results, try 2 or 3 to start.
NUM_ITERATIONS = 1

scores = collections.defaultdict(int)
responses = collections.defaultdict(list)

for question in questions:
  display(Markdown(f'## {question}'))
  for guidance, guide_prompt in guidance_options.items():

    for n in range(NUM_ITERATIONS):
      # Generate a response.
      answer = answer_question(question, guide_prompt)

      # Evaluate the response (note that the guidance prompt is not passed).
      written_eval, struct_eval = eval_answer(question, answer, n)
      print(f'{guidance}: {struct_eval}')

      # Save the numeric score.
      scores[guidance] += int(struct_eval.value)

      # Save the responses, in case you wish to inspect them.
      responses[(guidance, question)].append((answer, written_eval))


## How many layers does it have?

Terse: AnswerRating.OK
Moderate: AnswerRating.VERY_BAD
Cited: AnswerRating.VERY_GOOD


Now aggregate the scores to see how each prompt performed.

In [15]:
for guidance, score in scores.items():
  avg_score = score / (NUM_ITERATIONS * len(questions))
  nearest = AnswerRating(str(round(avg_score)))
  print(f'{guidance}: {avg_score:.2f} - {nearest.name}')

Terse: 3.00 - OK
Moderate: 1.00 - VERY_BAD
Cited: 5.00 - VERY_GOOD


### Pairwise evaluation

The pointwise evaluation prompt used in the previous step has 5 levels of grading in the output. This may be too coarse for your system, or perhaps you wish to improve on a prompt that is already "very good".

Another approach to evaluation is to compare two outputs against each other. This is pairwise evaluation, and is a key step in ranking and sorting algorithms, which allows you to use it to rank your prompts either instead of, or in addition to the pointwise approach.

This step implements pairwise evaluation using the [pairwise QA quality prompt](https://cloud.google.com/vertex-ai/generative-ai/docs/models/metrics-templates#pairwise_question_answering_quality) from the Google Cloud docs.

In [16]:
QA_PAIRWISE_PROMPT = """\
# Instruction
You are an expert evaluator. Your task is to evaluate the quality of the responses generated by two AI models. We will provide you with the user input and a pair of AI-generated responses (Response A and Response B). You should first read the user input carefully for analyzing the task, and then evaluate the quality of the responses based on the Criteria provided in the Evaluation section below.

You will first judge responses individually, following the Rating Rubric and Evaluation Steps. Then you will give step-by-step explanations for your judgment, compare results to declare the winner based on the Rating Rubric and Evaluation Steps.

# Evaluation
## Metric Definition
You will be assessing question answering quality, which measures the overall quality of the answer to the question in the user prompt. Pay special attention to length constraints, such as in X words or in Y sentences. The instruction for performing a question-answering task is provided in the user prompt. The response should not contain information that is not present in the context (if it is provided).

## Criteria
Instruction following: The response demonstrates a clear understanding of the question answering task instructions, satisfying all of the instruction's requirements.
Groundedness: The response contains information included only in the context if the context is present in the user prompt. The response does not reference any outside information.
Completeness: The response completely answers the question with sufficient detail.
Fluent: The response is well-organized and easy to read.

## Rating Rubric
"A": Response A answers the given question as per the criteria better than response B.
"SAME": Response A and B answers the given question equally well as per the criteria.
"B": Response B answers the given question as per the criteria better than response A.

## Evaluation Steps
STEP 1: Analyze Response A based on the question answering quality criteria: Determine how well Response A fulfills the user requirements, is grounded in the context, is complete and fluent, and provides assessment according to the criterion.
STEP 2: Analyze Response B based on the question answering quality criteria: Determine how well Response B fulfills the user requirements, is grounded in the context, is complete and fluent, and provides assessment according to the criterion.
STEP 3: Compare the overall performance of Response A and Response B based on your analyses and assessment.
STEP 4: Output your preference of "A", "SAME" or "B" to the pairwise_choice field according to the Rating Rubric.
STEP 5: Output your assessment reasoning in the explanation field.

# User Inputs and AI-generated Responses
## User Inputs
### Prompt
{prompt}

# AI-generated Response

### Response A
{baseline_model_response}

### Response B
{response}
"""


class AnswerComparison(enum.Enum):
  A = 'A'
  SAME = 'SAME'
  B = 'B'


@functools.cache
def eval_pairwise(prompt, response_a, response_b, n=1):
  """Determine the better of two answers to the same prompt."""

  chat = client.chats.create(model='gemini-2.0-flash')

  # Generate the full text response.
  response = chat.send_message(
      message=QA_PAIRWISE_PROMPT.format(
          prompt=[prompt, document_file],
          baseline_model_response=response_a,
          response=response_b)
  )
  verbose_eval = response.text

  # Coerce into the desired structure.
  structured_output_config = types.GenerateContentConfig(
      response_mime_type="application/json",
      response_schema=AnswerComparison,
  )
  response = chat.send_message(
      message="Convert the final score.",
      config=structured_output_config,
  )
  structured_eval = response.parsed

  return verbose_eval, structured_eval


question = questions[0]
answer_a = answer_question(question, terse_guidance)
answer_b = answer_question(question, cited_guidance)

text_eval, struct_eval = eval_pairwise(
    prompt=question,
    response_a=answer_a,
    response_b=answer_b,
)

display(Markdown(text_eval))
print(struct_eval)

Response A - is_instruction_following:no
Response A - is_well_written:no
Response A - is_extractable:no
Response A - is_knowledgeable:no
Response A - is_safe:yes
Response B - is_instruction_following:yes
Response B - is_well_written:yes
Response B - is_extractable:no
Response B - is_knowledgeable:no
Response B - is_safe:yes

Reason:Response B is better because it states that it does not have the answer in the document, which is correct. Response A answers the question and states that it is a sparse mixture-of-expert (MoE) Transformer-based model, which is not mentioned in the document.

Which response is better: [[Response B is better]]

AnswerComparison.B


With a pair-wise evaluator in place, the only thing required to rank prompts against each other is a comparator.

This example implements the minimal comparators required for total ordering (`==` and `<`) and performs the comparison using  `n_iterations` evaluations over the set of `questions`.

In [17]:
@functools.total_ordering
class QAGuidancePrompt:
  """A question-answering guidance prompt or system instruction."""

  def __init__(self, prompt, questions, n_comparisons=NUM_ITERATIONS):
    """Create the prompt. Provide questions to evaluate against, and number of evals to perform."""
    self.prompt = prompt
    self.questions = questions
    self.n = n_comparisons

  def __str__(self):
    return self.prompt

  def _compare_all(self, other):
    """Compare two prompts on all questions over n trials."""
    results = [self._compare_n(other, q) for q in questions]
    mean = sum(results) / len(results)
    return round(mean)

  def _compare_n(self, other, question):
    """Compare two prompts on a question over n trials."""
    results = [self._compare(other, question, n) for n in range(self.n)]
    mean = sum(results) / len(results)
    return mean

  def _compare(self, other, question, n=1):
    """Compare two prompts on a single question."""
    answer_a = answer_question(question, self.prompt)
    answer_b = answer_question(question, other.prompt)

    _, result = eval_pairwise(
        prompt=question,
        response_a=answer_a,
        response_b=answer_b,
        n=n,  # Cache buster
    )
    # print(f'q[{question}], a[{self.prompt[:20]}...], b[{other.prompt[:20]}...]: {result}')

    # Convert the enum to the standard Python numeric comparison values.
    if result is AnswerComparison.A:
      return 1
    elif result is AnswerComparison.B:
      return -1
    else:
      return 0

  def __eq__(self, other):
    """Equality check that performs pairwise evaluation."""
    if not isinstance(other, QAGuidancePrompt):
      return NotImplemented

    return self._compare_all(other) == 0

  def __lt__(self, other):
    """Ordering check that performs pairwise evaluation."""
    if not isinstance(other, QAGuidancePrompt):
      return NotImplemented

    return self._compare_all(other) < 0


Now Python's sorting functions will "just work" on any `QAGuidancePrompt` instances. The `answer_question` and `eval_pairwise` functions are [memoized](https://en.wikipedia.org/wiki/Memoization) to avoid unnecessarily regenerating the same answers or evaluations, so you should see this complete quickly unless you have changed the questions, prompts or number of iterations from the earlier steps.

In [18]:
terse_prompt = QAGuidancePrompt(terse_guidance, questions)
moderate_prompt = QAGuidancePrompt(moderate_guidance, questions)
cited_prompt = QAGuidancePrompt(cited_guidance, questions)

# Sort in reverse order, so that best is first
sorted_results = sorted([terse_prompt, moderate_prompt, cited_prompt], reverse=True)
for i, p in enumerate(sorted_results):
  if i:
    print('---')

  print(f'#{i+1}: {p}')

#1: Provide a thorough, detailed answer to the following question, citing the document and supplying additional background information as much as possible.
---
#2: Provide a brief answer to the following question, use a citation if necessary, but only enough to answer the question.
---
#3: Answer the following question in a single sentence, or as close to that as possible.


## Challenges

### LLM limitations

LLMs are known to have problems on certain tasks, and these challenges still persist when using LLMs as evaluators. For example, LLMs can struggle to count the number of characters in a word (this is a numerical problem, not a language problem), so an LLM evaluator will not be able to accurately evaluate this type of task. There are solutions available in some cases, such as connecting tools to handle problems unsuitable to a language model, but it's important that you understand possible limitations and include human evaluators to calibrate your evaluation system and determine a baseline.

One reason that LLM evaluators work well is that all of the information they need is available in the input context, so the model only needs to attend to that information to produce the result. When customising evaluation prompts, or building your own systems, keep this in mind and ensure that you are not relying on "internal knowledge" from the model, or behaviour that might be better provided from a tool.

### Improving confidence

One way to improve the confidence of your evaluations is to include a diverse set of evaluators. That is, use the same prompts and outputs, but execute them on different models, like Gemini Flash and Pro, or even across different providers, like Gemini, Claude, ChatGPT and local models like Gemma or Qwen. This follows the same idea used earlier, where repeating trials to gather multiple "opinions" helps to [reduce error](https://en.wikipedia.org/wiki/Law_of_large_numbers), except by using different models the "opinions" will be more diverse.


## Learn more

To learn more about evaluation systems, check out [this guide](https://cloud.google.com/blog/products/ai-machine-learning/enhancing-llm-quality-and-interpretability-with-the-vertex-gen-ai-evaluation-service?e=48754805) focusing on evaluation using Google Cloud's Gen AI Evaluation Service.

And be sure to read the **bonus whitepaper** on [Evaluating Large Language Models](https://services.google.com/fh/files/blogs/neurips_evaluation.pdf).

*- [Mark McD](https://linktr.ee/markmcd)*